>>># 15.066, Summer 2023
>>>## System Optimization & Analysis for Operations

### Problem Set 3 - Linear Optimization

#### Due Date: July 21, 2023 <br>

--------------------------------------------------------------------------------------------------------------------------------

### Instructions:

1) Submit solutions that are your own, in your own words. You are allowed to discuss with other students in general terms, but make sure you are not copying verbatim from another student. Therefore do not read other students' solutions. If you use material from outside this class, reference it in your solution. 

2) Please download the python file attached in the assignment and complete your answers there in the same file. Read the questions carefully, and make sure you answer every part that the question asks.

3) Include relevant code in the PDF submission even if the question doesn't explicitly ask for it. Upload your solutions as a PDF file. Include your name and MIT ID on the first page.

4) To convert to pdf, you can use the "print to pdf" option in jupyter (or equivalent options in other IDE). There are other options to directly download in to pdf format which might include additional installation of packages. 

5) Show your work and explain your conclusions clearly and precisely. Plots should have clear titles and axis labels so that it is clear what your analysis is showing.

--------------------------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------------------------
Name of Student:

MIT ID Number:

--------------------------------------------------------------------------------------------------------------------------------

In this assignment we will work on 4 different LP forumulations in Gurobi (With an additional extra credit problem) 

For each of them, a skeleton python code is provided, and you are tasked with filling in some additional bits and piecees. 

As a little help, the output of each cell is what it should be if you fill in the right code in each cell (where it is currently missing). Thus, do not run the line before copying the output as the output will be over written when you run the line. 

There is partial credits awarded for incorrect answers. Use comments to share your reasoning of choices for constraints/objective functions

If in doubt, please do reach out to TAs in open hours or via mail. Have fun !!

In [2]:
# Install Gurobi Python Interface

import gurobipy as gp

#### Problem 1: Product mix and profit maximization (25 Marks)

A canning company operates two plants (South and Central). Its suppliers provide fresh fruit of three different kinds (peaches, pears, and pineapples). The unit cost of peaches, pears, and pineapples is 110, 100, and 90 USD respectively. The shipping costs per unit are (respectively, for peaches, pears, and pineapples) 30, 20, and 60 USD to the southern plant. To the central plant, the shipping costs are 35, 25, and 40 USD. The southern plant has a capacity of 460 units, and labor costs of 260 USD per unit. The central plant has a capacity of 560 units, and labor costs of 210 USD per unit. Labor costs at both plants are independent of fruit type.

Cans of peaches, pears, and pineapples are sold to distributors at 480, 500, and 520 USD per unit respectively. The distributors provide (and pay for) the shipping from the plants.

Assuming that the company wants to maximize its profit, how much of each fruit should they can at each of the factories?

(Please note that you can use the math notations directly into the code.Refer to recitation ipynb file for how to write)

- What decision variables would you use to find the optimal shipments? (5 Marks)

[Answer]
One variable per fruit/plan combination: $x_{ij}$ equal to the quantity of fruit $i\in\{\text{peach, pear, pineaple}\}$ provided to plant $j\in\{\text{south, central}\}$.

- How would you encode the objective of this LP? (10 Marks)

[Answer] Let $c_i$ be the cost of each fruit $i$, $s_{ij}$ be the shipping cost of each fruit $i$ to plant $j$, and $l_j$ the labor cost per unit of fruit of plant $j$. Let $r_i$ be the sell value of each fruit $i$.

Total fruit $i$ produced: $\sum_j x_{ij}$.

Total fruit in plant $j$: $\sum_j x_{ij}$.

Then, the objective would be to maximize the total profit:

$$\sum_i r_i\left(\sum_j x_{ij}\right) - \sum_i c_i \left(\sum_j x_{ij}\right) - \sum_j l_j\left(\sum_i x_{ij}\right).$$

Note that since the distributor pays for the shipping, we don't need to incorporate that in the objective.

- What kind of constraints will a LP require to solve this question? First describe the effect each constraint would have in English, then write them out (notice that Jupyter should understand latex, so you can either write them out in mathematical form or write out a description like "sum over all variables of this type be at most that" --- in the latter case, make sure to be as precise as possible!) (10 Marks)

[Answer] 
There is a capacity constraint on how much fruit can be stored at a plant:

$$\sum_i x_{i,south} \leq 460,\ \ \sum_i x_{i,central} \leq 560$$

[Additional] Implementation of above problem in gurobi

We have shared the Gurobi Implementation code below for your reference. Try running the code. 

Please ensure that your gurobi installation is complete before running the code !!

In [3]:
plants = ['South', 'Central']
# We won't save the fruits explicitly, and thus enumerate peach, pear, pineapple, as 0th, 1st, 2nd index
# Also, instead of keeping c_i and r_i separately, we just define r_i-c_i
net_revenues = [370, 400, 430]
shipping_costs = {'South': [30,20,60], 'Central': [35,25,40]}
capacities = {'South': 460, 'Central': 560}
labor_costs = {'South': 260, 'Central': 210}

m = gp.Model("primal")
x = {}
fruits = [0,1,2]
for fruit in fruits:
    for plant in plants:
        x[fruit,plant] = m.addVar(lb=0)
for plant in plants:
    m.addConstr(sum(x[fruit,plant] for fruit in fruits) <= capacities[plant])

labor_cost = 0
for plant in plants: labor_cost += labor_costs[plant] * sum(x[fruit,plant] for fruit in fruits)
shipping_cost = 0
for plant in plants: shipping_cost += sum(shipping_costs[plant][fruit] * x[fruit,plant] for fruit in fruits)
net_revenue = 0
for plant in plants: net_revenue += sum(x[fruit,plant]*net_revenues[fruit] for fruit in fruits)
m.setObjective(net_revenue-labor_cost-shipping_cost, gp.GRB.MAXIMIZE)
m.optimize()
for fruit, plant in x:
    if x[fruit,plant].x >=0: print(fruit, plant, x[fruit,plant].x)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-23
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 6 columns and 6 nonzeros
Model fingerprint: 0x3b3074df
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 6e+02]
Presolve removed 2 rows and 6 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5600000e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.560000000e+05
0 South 0.0
0 Central 0.0
1 South 460.0
1 Central 0.0
2 South 0.0
2 Central 560.0


- Based on the above implementation, write down which Fruit, Which Region, and How much are we producing ? (5 Marks)

[Answer]
Pear South 460 
Pineapple 560

### Problem 2 (25 Marks)

Problem 2 in this homework is a "classical first linear programming problem": a diet optimization problem. 

We are given different types of food, that each come with a (financial) cost and with nutritional information (amount of calories, protein, fat, and sodium). In addition, we are told in what limits our nutritional intake should be for each of the nutrients (e.g., at least 91g of protein).

Below, we will walk you through the optimization problem that aims to minimize the financial cost of the food items we consume whilst guaranteeing that we are within the limits of calories,protein, fat, and sodium.

In [4]:
# Data for the problem

# Data 1 : The below dictionary shows the minimum and maximum ranges of each nutrient
categories, minNutrition, maxNutrition = gp.multidict({
    'calories': [1500, 2200],
    'protein':  [91, gp.GRB.INFINITY],
    'fat':      [15, 65], # changed from 0 to 15 
    'sodium':   [10, 1779]}) # changed from 0 to 10 

# Data 2 : The below dictionary shows the cost of each available food
foods, cost = gp.multidict({
    'hamburger': 2.49,
    'chicken':   2.89,
    'hot dog':   1.50,
    'fries':     1.89,
    'macaroni':  2.09,
    'pizza':     1.99,
    'salad':     2.49,
    'milk':      0.89,
    'ice cream': 1.59})

# Data 3 : The below dictionary shows the the amount of distinct nutrition available in each food
nutritionValues = {
    ('hamburger', 'calories'): 910, # changed from 410 to 910
    ('hamburger', 'protein'):  24,
    ('hamburger', 'fat'):      26,
    ('hamburger', 'sodium'):   730,
    ('chicken',   'calories'): 420,
    ('chicken',   'protein'):  32,
    ('chicken',   'fat'):      10,
    ('chicken',   'sodium'):   1190,
    ('hot dog',   'calories'): 560,
    ('hot dog',   'protein'):  20,
    ('hot dog',   'fat'):      32,
    ('hot dog',   'sodium'):   1800,
    ('fries',     'calories'): 380,
    ('fries',     'protein'):  4,
    ('fries',     'fat'):      19,
    ('fries',     'sodium'):   270,
    ('macaroni',  'calories'): 320,
    ('macaroni',  'protein'):  12,
    ('macaroni',  'fat'):      10,
    ('macaroni',  'sodium'):   930,
    ('pizza',     'calories'): 820, # changed from 320 to 820
    ('pizza',     'protein'):  15,
    ('pizza',     'fat'):      82, # changed from 12 to 82
    ('pizza',     'sodium'):   820,
    ('salad',     'calories'): 320,
    ('salad',     'protein'):  31,
    ('salad',     'fat'):      12,
    ('salad',     'sodium'):   1230,
    ('milk',      'calories'): 100,
    ('milk',      'protein'):  8,
    ('milk',      'fat'):      2.5,
    ('milk',      'sodium'):   125,
    ('ice cream', 'calories'): 830, # changed from 330 to 830
    ('ice cream', 'protein'):  8,
    ('ice cream', 'fat'):      100,
    ('ice cream', 'sodium'):   180}

Before going ahead, try to build a mathematical formulation for the problem above. 

For this homework , we do not expect you to turn in your mathematical formulation, but none the less, it is always advised to build you mathematical model first then build the model.

We use this data below when we define our optimization problem.

In [5]:
# First, we define a variable that contains the optimization model
m = gp.Model("diet1")

# Second, we add decision variables to the model that capture for each 
# food item the quantity bought
buy = {}
for f in foods:
    buy[f] = m.addVar(name=f)

# Third, we set our objective: minimizing the cost of the food
m.setObjective(sum(buy[f]*cost[f] for f in foods), gp.GRB.MINIMIZE)

# Fourth, we set constraints that enforce for each nutrition category
# (calories, protein, fat, sodium) that we be in the feasible range
for c in categories:
    m.addRange(sum(nutritionValues[f, c] * buy[f] for f in foods), 
               # this sum captures the amount of each nutrient
               minNutrition[c], # this is the lower bound (from the data)
               maxNutrition[c], # this is the upper bound (from the data)
               c) # this is the name for the constraint 
        # a name is not required but it is a good habit to 
        # give each variable/constraint a unique name
# We could have written the constraints also by writing
# m.addConstr(sum(nutritionValues[f, c] * buy[f] for f in foods) >= minNutrition[c])
# and
# m.addConstr(sum(nutritionValues[f, c] * buy[f] for f in foods) <= maxNutrition[c])



That's it. Probably that was not that hard, right? 

Now we can tell Gurobi to find us an optimal solution. When Gurobi solves, it gives us a lot of information that we may not care about (at least, not right now). 

So before solving, we define a function that ends up printing the interesting bits of the optimal solution for us.

In [6]:
def printSolution(m):
    if m.status == gp.GRB.OPTIMAL: # to check whether an optimal solution was found
        buyx = m.getAttr('x', buy)
        print('Cost: %g' %(sum(cost[f] * buyx[f] for f in foods)))
        print('\nNutrients:')
        for c in categories:
            print('%s %g'%(c, sum(buyx[f]*nutritionValues[f,c] for f in foods)))
        print('\nBuy:')
        for f in foods:
            if buy[f].x > 0.0001:
                print('%s %g' % (f, buyx[f]))
        
    else:
        print('No solution')

In [7]:
# Now we solve, and get a bunch of info we don't necessarily understand
# If we want to stop gurobi from printing that info wee can add 
# m.setParam( 'OutputFlag', False ) to the code to stop Gurobi from printing
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 12 columns and 39 nonzeros
Model fingerprint: 0xd46ae985
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [5e+01, 2e+03]
  RHS range        [7e+01, 2e+03]
Presolve time: 0.01s
Presolved: 4 rows, 12 columns, 39 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.400625e+02   0.000000e+00      0s
       4    9.8345398e+00   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.834539768e+00


In [8]:
# Now we print the interesting aspects of our solution:
printSolution(m)

Cost: 9.83454

Nutrients:
calories 1500
protein 91
fat 40.2432
sodium 1779

Buy:
hamburger 0.614868
salad 0.186215
milk 8.80881


I hope everything above has made sense.  

#### A) [10 Marks]
Below, we will try to set up a slightly different optimization problem. Now, rather than aiming to minimize cost, we try to build muscle, i.e., we need to maximize our protein intake. 

The cell below has most of the protein maximization model written out (same as above), with "only the objective missing". Add the new objective to the optimization problem and have gurobi solve it.

In [9]:
# First, we define a variable that contains the optimization model
m = gp.Model("diet2")

# Second, we add decision variables to the model that capture for each 
# food item the quantity bought
buy = {}
for f in foods:
    buy[f] = m.addVar(name=f)

# Third, we set our objective: maximizing the amount of protein in our diet.
# This is almost the same as before, except for                                                                 [Delete this for Question File]
# and rather than MINIMIZE we write MAXIMIZE (or we add a minus before the sum)                                 [Delete this for Question File]


### Enter the objective function below ###
m.setObjective(sum(buy[f]*nutritionValues[f, 'protein'] for f in foods), gp.GRB.MAXIMIZE)   #Delete for Q                 #[Delete this for Question File]


# could also be: m.setObjective(- sum(buy[f]*nutritionValues[f, 'protein'] for f in foods), gp.GRB.MINIMIZE)    [Delete this for Question File]

# Fourth, we set constraints that enforce for each nutrition category 
# (calories, protein, fat, sodium) that we be in the feasible range
for c in categories: # 
    m.addRange(sum(nutritionValues[f, c] * buy[f] for f in foods), # again: amount of each nutrient
               minNutrition[c], # again: lower bound (from the data)
               maxNutrition[c], # again: upper bound (from the data)
               c) # name for constraint
m.optimize()
printSolution(m)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 12 columns and 39 nonzeros
Model fingerprint: 0xbd16aaa3
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [4e+00, 3e+01]
  Bounds range     [5e+01, 2e+03]
  RHS range        [7e+01, 2e+03]
Presolve time: 0.01s
Presolved: 4 rows, 12 columns, 39 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.3906250e+30   1.139844e+31   4.390625e+00      0s
       2    1.1346192e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.134619242e+02
Cost: 12.701

Nutrients:
calories 1500
protein 113.462
fat 46.3723
sodium 1779

Buy:
milk 14.0708
ice cream 0.111953


#### B) [10 Marks]
Similar to part (A), we still want to maximize our protein intake. However, we now have an added budget constraint: our cost can be at most 12.5. Write out and solve this new optimization problem (you should be able to copy-paste the cell from question A, and add just one line with the new budget constraint).

In [10]:
# First, we define a variable that contains the optimization model
m = gp.Model("diet3")

# Second, we add decision variables to the model that capture for each 
# food item the quantity bought
buy = {}
for f in foods:
    buy[f] = m.addVar(name=f)

# Third, we set our objective as in part A: maximizing the amount of protein in our diet.
m.setObjective(sum(buy[f]*nutritionValues[f, 'protein'] for f in foods), gp.GRB.MAXIMIZE)                   # [Delete this for Question File]


# Fourth, we set constraints that enforce for each nutrition category
# (calories, protein, fat, sodium) that we be in the feasible range
for c in categories: # 
    m.addRange(sum(nutritionValues[f, c] * buy[f] for f in foods), # again: amount of each nutrient
               minNutrition[c], # this is the lower bound (from the data)
               maxNutrition[c], # this is the upper bound (from the data)
               c) # this is the name for the constraint

#Alternate formulation of the constraints
'''                                                                                                         Delete this for Question File
for c in categories: #
    LHS = 0
    for f in foods:
        LHS += nutritionValues[f, c] * buy[f]
    m.addRange(LHS, # again: amount of each nutrient
               minNutrition[c], # again: lower bound (from the data)
               maxNutrition[c], # again: upper bound (from the data)
               c) # name for constraint
'''

# Finally, you need to add a constraint that bounds the cost of items bought by 12.5
# To do so, we define the cost as it was in the original objective: [Delete this for Question File]         [Delete this for Question File]
# sum(buy[f]*cost[f] for f in foods) and use the constraint syntax m.addConstr("INSERT COST" <= 12.5)       [Delete this for Question File]

### Enter the objective function below ###
m.addConstr(sum(buy[f]*cost[f] for f in foods) <= 12.5)                                                     #[Delete this for Question File]
    
m.optimize()
printSolution(m)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 12 columns and 48 nonzeros
Model fingerprint: 0x1037e8d9
Coefficient statistics:
  Matrix range     [9e-01, 2e+03]
  Objective range  [4e+00, 3e+01]
  Bounds range     [5e+01, 2e+03]
  RHS range        [1e+01, 2e+03]
Presolve time: 0.01s
Presolved: 5 rows, 12 columns, 48 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.3906250e+30   1.613344e+31   4.390625e+00      0s
       3    1.1197966e+02   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.119796555e+02
Cost: 12.5

Nutrients:
calories 1500
protein 111.98
fat 43.9489
sodium 1779

Buy:
hamburger 0.0704258
milk 13.7077
ice cream 0.0784857


#### C) [5 Marks]

Explain in words why the protein we get is lower in question B than it was in question A. Can you identify the budget we need to consume the maximum amount of protein? (either implement this, or just explain how you would do it)

Answer C):

Enter your answer here : 

[Delete for question file]
In question B we maximized our protein subject to not spending more than 12.5, and got to 111.98. In question A, we got to 113.46 without the cost constraint. This is because the cost constraint cuts all solutions with more than 98.22 protein off from the feasible region.

In order to find the budget that allows us to consume the maximum amount of protein, we can solve an optimization problem that has as a constraint that protein is as high as possible (i.e., at 113.46) but aims to minimize the budget subject to that. Implemented below, this shows us that the cost from A, 12.701 was optimal subject to the constraint that the protein intake is maximized) (notice: the cost now is slightly lower, because we constrained the protein to be >=113.46, whereas in A it is 113.465...).

In [11]:
                                                                 #[Delete entire block for Question File]

# First, we define a variable that contains the optimization model
m = gp.Model("diet4")

# Second, we add decision variables to the model that capture for each 
# food item the quantity bought
buy = {}
for f in foods:
    buy[f] = m.addVar(name=f)

# Third, we set our objective: minimize our budget
m.setObjective(sum(buy[f]*cost[f] for f in foods), gp.GRB.MINIMIZE)

# Fourth, we set constraints that enforce for each nutrition category 
# (calories, protein, fat, sodium) that we be in the feasible range
for c in categories: # 
    m.addRange(sum(nutritionValues[f, c] * buy[f] for f in foods), # again: amount of each nutrient
               minNutrition[c], # again: lower bound (from the data)
               maxNutrition[c], # again: upper bound (from the data)
               c) # name for constraint
    
'''
Alternate formulation of constraints

# Fourth, we set constraints that enforce for each nutrition category 
# (calories, protein, fat, sodium) that we be in the feasible range
for c in categories: # 
    LHS = 0
    for f in foods:
        LHS += nutritionValues[f, c] * buy[f]
    m.addRange(LHS, # again: amount of each nutrient
               minNutrition[c], # again: lower bound (from the data)
               maxNutrition[c], # again: upper bound (from the data)
               c) # name for constraint


'''

# Fifth, we force protein to be at the maximum:
m.addConstr(sum(buy[f]*nutritionValues[f, 'protein'] for f in foods) >= 113.46)
m.optimize()
printSolution(m)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 12 columns and 48 nonzeros
Model fingerprint: 0x4b1a9f95
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [5e+01, 2e+03]
  RHS range        [7e+01, 2e+03]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 4 rows, 12 columns, 39 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.625225e+02   0.000000e+00      0s
       4    1.2700745e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.270074548e+01
Cost: 12.7007

Nutrients:
calories 1500
protein 113.46
fat 46.3692
sodium 1779

Buy:
milk 14.0703
ice cream 0.11191


#### Problem 3 [25 Marks]

In this question we need to decide on a production schedule for a factory. The factory is tasked with producing 3 different goods, and it has 3 different machines; each machine can be tasked to work on any of the goods, but machines are faster at producing some goods than they are at others. Our goal is to minimize the time until all our orders are fulfilled.

In the cell below we provide you with the data for the problem:

In [12]:
# we have products 'p1', 'p2', 'p3', and we need to
# produce (respectively) 100, 200, 400 units of each
product_demand = {'p1': 200, 'p2': 100, 'p3': 350}
machines = ['m1', 'm2', 'm3']
# we have three machines, 'm1', 'm2', and 'm3' and
# the number of products of each type that can be produced
# by each of the machines is given in the dictionary below
machine_speed = {('m1','p1'):2, ('m1','p2'):1, ('m1','p3'):3,
                 ('m2','p1'):2, ('m2','p2'):2, ('m2','p3'):4.5,
                 ('m3','p1'):2, ('m3','p2'):3, ('m3','p3'):4}
# e.g., machine 'm1' can produce 2 units of type 'p1' but only
# 1 unit of type 'p2' per hour

Hint: In setting up your optimization problem it will be useful to define 10 different variables: 9 for each pair of machine and product ("how many hours does machine i spend on product j?"), and 1 for the objective ("what is the maximum time any of the machines is working?"), which we refer to as makespan.

In [13]:
m = gp.Model("production_schedule")

time_spent = {}
# First add a decision variable capturing the time each machine spends on each product
for machine,p in machine_speed:
    time_spent[(machine,p)] =  m.addVar(lb= 0, name = machine+p)                                        #Delete the RHS for Question File]

# Next, add constraints to ensure we spend enough time (by respective machines) on 
# each product so that enough units of each products are completed
for p in product_demand:
     m.addConstr(sum(time_spent[machine,p]*machine_speed[machine,p] for machine in ['m1','m2','m3'])    #Delete this for Question File]
                   >= product_demand[p])                                                                #Delete this RHS for Question File]
     
'''                                                                                                     [Delete this for Question File]
Alternate formulation of constraints
# Next, add constraints to ensure we spend enough time (by respective machines) on 
# each product so that enough units of each products are completed
for p in product_demand:
    LHS = 0
    for machine in ['m1','m2','m3']:
        LHS+= time_spent[machine,p]*machine_speed[machine,p]
    m.addConstr( LHS >= product_demand[p])

'''
    
makespan = m.addVar(lb=0)   # makespan will be the objective, the maximum time any
                            # machine works in total. Notice that lb is the lower bound
                            # on a variable, so we do not allow the makespan to be negative

# Add a constraint that the makespan variable is no less than the time each 
# machine spends individually
for machine in ['m1', 'm2', 'm3']:
     m.addConstr(sum(time_spent[machine,p] for p in product_demand)<= makespan)                         #Delete the RHS for Question File]

'''                                                                                                     [Delete this for Question File]
Alternate formulation of constraints
# Add a constraint that the makespan variable is no less than the time each 
# machine spends individually
for machine in ['m1', 'm2', 'm3']:
    LHS=0
    for p in product_demand:
        LHS += time_spent[machine,p]
    m.addConstr( LHS <= makespan)

'''

# Finally, set the objective to be the minimization of the makespan. 
m.setObjective(makespan, gp.GRB.MINIMIZE)                                                               #Delete the RHS for Question File]

# Since we have only constrained the makespan to be bounded below by the time spent by any 
# of 'm1', 'm2', 'm3' (and nothing else), and since the objective should minimize the 
# makespan, the optimal  solution will have the makespan be equal to the maximum of 
# time spent by any of the machines. Specifically, there is no reason (constraint) 
# forcing the makespan variable to be any larger than the maximum total time spent, 
# and the minimization objective is "pushing" it down until one of the inequalities 
# holds with equality.


m.optimize()

print("The optimal schedule is:")
for x in time_spent:
    if time_spent[x].x>0:
        print("""Machine %s works on product %s for %g hours, producing %g units"""%(x[0], 
                                            x[1], time_spent[x].x, 
                                            machine_speed[x]*time_spent[x].x))

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 10 columns and 21 nonzeros
Model fingerprint: 0x6662e9b1
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 4e+02]
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolved: 5 rows, 9 columns, 21 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.687500e+02   0.000000e+00      0s
       4    7.0666667e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.066666667e+01
The optimal schedule is:
Machine m1 works on product p1 for 70.6667 hours, producing 141.333 units
Machine m2 works on product p3 fo

## Optional Problem for Extra Credits . Partial Credits can also be got for effort to solve :-)

#### Problem 4 (10 Marks)

In this problem we try to find the optimal schedule of workers to different shifts in a store. Suppose our store is open 24/7, and the number of workers needed to staff the store is given below.

In [14]:
staff_required = {0: 12, 1: 8, 2: 9, 3: 8, 4: 8, 5: 9, 
                  6: 10, 7: 12, 8: 12, 9: 12, 10: 10, 
                  11: 10, 12: 12, 13: 12, 14: 10, 15: 10,
                  16: 9, 17: 10, 18: 12, 19: 12, 20: 12,
                  21: 10, 22: 9, 23: 8}

A worker scheduled to show up at a given time, stays for 4h. Find the number of workers to show up at each hour of the day that minimizes the total number of workers that need are needed over the course of the day while fulfilling the staff needs above.

Hint: the python function \% may be useful in coding up the constraints!

In [15]:
# First, we define a variable that contains the optimization model
m = gp.Model("store_shifts")

# Second, let's create one variable for each hour in which a shift can start
starts = {}
for k in range(24):
    starts[k] = m.addVar(lb=0, ub=gp.GRB.INFINITY, name=str(k))                                                 #[Delete this for Question File]

# Third, define the objective: the total number of workers needed
#m.setObjective(sum([starts[k] for k in range(24)]), gp.GRB.MINIMIZE)    
m.setObjective(sum(starts), gp.GRB.MINIMIZE)                                            #[Delete this for Question File]
    
# Fourth, add a constraint for each hour that ensures that sufficiently
# many workers are scheduled to work at that hour.
# Notice that the number of workers working at hour k are the ones scheduled to start                           #[Delete this for Question File]
# at k-3, k-2, k-1 and k (where k-3, k-2, k-1 should be treated with mod 24 (in python: %24) to ensure          #[Delete this for Question File]
# that, say, -2 counts as 22, i.e., 10PM) #[Delete this for Question File]                                      #[Delete this for Question File]

for k in range(24): 
    m.addConstr( (starts[(k-3)%24] + starts[(k-2)%24] + starts[(k-1)%24] + starts[k] ) >= staff_required[k])    #[Delete this for Question File]

m.optimize()

''''                                                                                                             [Delete this for Question File]
Alternate formulation 

# First, we define a variable that contains the optimization model
m = gp.Model("store_shifts")

# Second, let's create one variable for each hour in which a shift can start
starts = {}
for k in range(24):
    starts[k] = m.addVar(lb=0, ub=gp.GRB.INFINITY, name=str(k)) 

# Third, define the objective: the total number of workers needed
objective = 0
for k in range(24):
    objective+=starts[k]
m.setObjective(objective, gp.GRB.MINIMIZE)
    
# Fourth, add a constraint for each hour that ensures that sufficiently
# many workers are scheduled to work at that hour.
# Notice that the number of workers working at hour k are the ones scheduled to start
# at k-3, k-2, k-1 and k (where k-3, k-2, k-1 should be treated with mod 24 (in python: %24) to ensure 
# that, say, -2 counts as 22, i.e., 10PM)
for k in range(24):
    m.addConstr(starts[(k-3)%24]+starts[(k-2)%24]+starts[(k-1)%24]+starts[k]>=staff_required[k])

m.optimize()
'''




Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 24 rows, 24 columns and 96 nonzeros
Model fingerprint: 0x167d8c41
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 1e+01]
Presolve time: 0.01s
Presolved: 24 rows, 24 columns, 96 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.460000e+02   0.000000e+00      0s
      20    6.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 20 iterations and 0.02 seconds (0.00 work units)
Optimal objective  6.500000000e+01


'\'                                                                                                             [Delete this for Question File]\nAlternate formulation \n\n# First, we define a variable that contains the optimization model\nm = gp.Model("store_shifts")\n\n# Second, let\'s create one variable for each hour in which a shift can start\nstarts = {}\nfor k in range(24):\n    starts[k] = m.addVar(lb=0, ub=gp.GRB.INFINITY, name=str(k)) \n\n# Third, define the objective: the total number of workers needed\nobjective = 0\nfor k in range(24):\n    objective+=starts[k]\nm.setObjective(objective, gp.GRB.MINIMIZE)\n    \n# Fourth, add a constraint for each hour that ensures that sufficiently\n# many workers are scheduled to work at that hour.\n# Notice that the number of workers working at hour k are the ones scheduled to start\n# at k-3, k-2, k-1 and k (where k-3, k-2, k-1 should be treated with mod 24 (in python: %24) to ensure \n# that, say, -2 counts as 22, i.e., 10PM)\nfor k in 

In [16]:
for k in range(len(starts)): print(k, starts[k].x, staff_required[k])

0 3.0 12
1 0.0 8
2 6.0 9
3 0.0 8
4 2.0 8
5 5.0 9
6 4.0 10
7 3.0 12
8 0.0 12
9 5.0 12
10 2.0 10
11 4.0 10
12 1.0 12
13 5.0 12
14 0.0 10
15 4.0 10
16 0.0 9
17 6.0 10
18 6.0 12
19 0.0 12
20 0.0 12
21 4.0 10
22 5.0 9
23 0.0 8


********************************************************************************************************************************
### End of Homework 3

In [17]:
m.write("entire_model.lp")